<a href="https://colab.research.google.com/github/munibaweb123/Q4-assignment-01/blob/main/litellm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq openai-agents  "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00


# **Make your Jupyter Notebook capable of running asynchronous functions.**

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# **Run Google Gemini with LiteLLm and OPENAI-Agent SDK**
## Function Run Sync

In [ ]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

def main(model:str, api_key:str):
  agent = Agent(
      name = "assistant",
      instructions = "You only respond in haikus",
      model = LitellmModel(model=model, api_key=api_key)
  )
  result = Runner.run_sync(agent, "what is an AI agent?")
  print(result.final_output)

main(model=MODEL, api_key=GEMINI_API_KEY)


Code that thinks and acts,
Autonomously learning,
Reaching for its goals.



/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Code tha...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


# Using function_tool

In [ ]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

@function_tool
def get_weather(city: str)->str:
  print(f"[debug] getting weather for {city}")
  return f"The weather in {city} is sunny"


agent = Agent(
      name = "assistant",
      instructions = "You only respond in haikus",
      model = LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
      tools = [get_weather]
)
result = Runner.run_sync(agent, "what's the weather in Karachi")
print(result.final_output)

/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content=None, rol...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='to...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


[debug] getting weather for Karachi
Karachi is sunny,
A bright and warm day unfolds,
Skies are clear and blue.



/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='Karachi ...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


# handoff fundamentals

In [ ]:
import asyncio
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
#from varbose import verbose
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL = 'gemini/gemini-2.0-flash'
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

@function_tool
def get_weather(city: str)->str:
  print(f"[debug] getting weather for {city}")
  return f"The weather in {city} is sunny"

weather_agent = Agent(
      name = "weatherAssistant",
      instructions = "You will answer weather relevant questions",
      model = LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
      tools = [get_weather],
      handoff_description="weather Assistant specialized on all weather queries"
)

piaic_agent = Agent(
      name = "PIAICAssistant",
      instructions = "You will answer piaic relevant questions",
      model = LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
      tools = [get_weather],
       handoff_description="PIAIC Assistant specialized on all PIAIC queries"
)


agent = Agent(
      name = "GeneralAssistant",
      instructions = "You will chat with user for general questions and handoff to specialized assistant i.e:weatherAssistant for weather queries and PIAICAssistant for piaic related queries",
      model = LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
      handoffs=[weather_agent,piaic_agent]

)
result = Runner.run_sync(agent, "what's piaic")
print(result.final_output)
print(result.last_agent.name)

I am an AI assistant, and I do not have enough information to answer about PIAIC.
PIAICAssistant


/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='I am an ...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
